## **Import Libraries**



In [5]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import csv
import math
import heapq
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import spacy

## **Load the data**

In [6]:
data = pd.read_csv('newdata_2.csv')
# print(tfidf['Name'])
# print(data)
# print(data.shape)

corpus = list(data['Corpus'])
# corpus_words = vectorizer.get_feature_names()
vectorizer1 = CountVectorizer()
word_vec = vectorizer1.fit_transform(corpus)
corpus_words = vectorizer1.get_feature_names()

# nlp_tool = spacy.load('en_core_web_sm')
# for text in corpus:
#     token_sentence = nlp_tool(text.lower())
#     print(token_sentence)
#     for word in token_sentence:
#         print(word)

print(len(corpus_words))
print(corpus_words)
# print
print('cute' in corpus_words)

1770
['abandonment', 'abit', 'able', 'abnormal', 'abnormality', 'absolute', 'absolutely', 'abuse', 'accept', 'accident', 'accidental', 'accordingly', 'act', 'action', 'active', 'actively', 'acupuncture', 'adapt', 'adjust', 'administer', 'admirably', 'adoptive', 'adorable', 'adult', 'adventure', 'adventurous', 'affection', 'affectionate', 'affectionately', 'affirm', 'afraid', 'aggression', 'aggressive', 'aggressively', 'agile', 'agree', 'ahead', 'aid', 'aimlessly', 'air', 'alas', 'aleck', 'alert', 'alertness', 'alfa', 'alike', 'allegedly', 'allergysharpei', 'alleviate', 'alongside', 'aloof', 'alpha', 'alright', 'amazing', 'ang', 'angel', 'animal', 'answer', 'anxiety', 'anxious', 'anytime', 'apartment', 'appeal', 'appear', 'appetite', 'apply', 'appreciate', 'apprehension', 'apprehensive', 'approx', 'aptly', 'arise', 'arm', 'arrival', 'arrive', 'artist', 'ash', 'ask', 'aspirate', 'assertive', 'assessment', 'assimilate', 'assistance', 'assume', 'assurance', 'assure', 'assured', 'attack', '

D:\Anaconda3\envs\py38gpu\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


## **Build Up the Language Processor**

In [7]:
def my_preprocessing(raw_sentence):
    nlp_tool = spacy.load('en_core_web_sm')
    token_sentence = nlp_tool(raw_sentence.lower())
    with open('./irrelevant_words.txt') as file:
        irrelevantlist = [stopword.replace('\n', '').lower() for stopword in file.readlines()]
#     new_sentence = [word for word in token_sentence if word not in irrelevantlist]
    
    preprocessed_sentence = []
    
    for token in token_sentence:
        if token.pos_ == "PUNCT" or token.is_stop == True or token.is_alpha == False or token.pos_ == "SYM":
            continue

        elif token.lemma_ in irrelevantlist or len(token)==1:
            continue
            
        else:
            preprocessed_sentence.append(token.lemma_)
            
    return preprocessed_sentence


## Basic functions

In [8]:
# Filter the dogs according to the hard constraints (gender, age and HDB approval)
def filter_dog(data,gender,age,hdb):
    filtered = []
    if hdb == 0:
        hdb_ = [0,1]
    else:
        hdb_ = [1]
    for i in range(335):
        if data.loc[i,'Gender'] in gender:
            if data.loc[i,'Age'] in age:
                if data.loc[i,'HDB'] in hdb_:
                    filtered.append(i)
    return filtered

# Test
filtered = filter_dog(data,[1],[2],[0])
print('Number of dogs after filterring: ' + str(len(filtered)))
print('Index of dogs after filtering:')
print(filtered)
# index = []
# for dog in filtered:
#     index.append(dog['Index'])
# print(index)

nofilter = filter_dog(data,[0,1],[0,1,2,3],[0,1])

Number of dogs after filterring: 11
Index of dogs after filtering:
[110, 160, 178, 179, 181, 205, 219, 233, 275, 296, 297]


In [9]:
# # Make a list of values of dogs to calculate cosine similarity
# def make_list(dog_index,hard_constraints,tfidf,soft_constraints):
#     value_list = []
#     if hard_constraints == True: 
#         value_list.append(data.loc[dog_index]['Gender'])
#         value_list.append(data.loc[dog_index]['Age'])
#         value_list.append(data.loc[dog_index]['HDB'])
#     if tfidf == True:
#         for i in range(1770):
#             value_list.append(data.loc[dog_index]['tfidf'+str(i)])
#     if soft_constraints == True:
#         for i in range(6):
#             value_list.append(data.loc[dog_index]['Tag'+str(i+3)])
# #     print(len(value_list))
#     return value_list

# dogs_value_list = []
# for i in range(335):
#     dogs_value_list.append(make_list(i,True,True,True))

In [30]:
def cosine_similarity(dogx,dogy,hard_constraint,soft_constraint,tfidf,user):
    numerator = 0
    denominator_x = 0
    denominator_y = 0
    if hard_constraint == True:
        numerator += dogx['Gender'] * dogy[indexy]['Gender']
        numerator += dogx[indexx]['Age'] * dogy[indexy]['Age']
        numerator += dogx[indexx]['HDB'] * dogy[indexy]['HDB']
        denominator_x += dogx[indexx]['Gender'] ** 2
        denominator_x += dogx[indexx]['Age'] ** 2
        denominator_x += dogx[indexx]['HDB'] ** 2
        denominator_y += dogy[indexx]['Gender'] ** 2
        denominator_y += dogy[indexx]['Age'] ** 2
        denominator_y += dogy[indexx]['HDB'] ** 2
    if soft_constraint == True:
        for i in range(6):
            if user == True:
                tagix = dogx['Tag'+str(i+3)]
                tagiy = dogy['Tag'+str(i+3)]
                if dogx['Tag'+str(i+3)] == 0 or dogy['Tag'+str(i+3)] != 0:
                    tagix = 0
                    tagiy = 0
            numerator += tagix * tagiy
            denominator_x += tagix ** 2
            denominator_y += tagiy ** 2
    if tfidf == True:
        for i in range(1770):
            numerator += dogx['tfidf'+str(i)] * dogy['tfidf'+str(i)]
            denominator_x += dogx['tfidf'+str(i)]**2
            denominator_y += dogy['tfidf'+str(i)]**2

    denominator_x = math.sqrt(denominator_x)
    denominator_y = math.sqrt(denominator_y)  
    if denominator_x*denominator_y == 0:
        return 0
    return (numerator/(denominator_x*denominator_y))

In [11]:
# # Calculation of cosine similarity between two dogs
# # For the soft constraints, if the value of one certain aspect for either dog is 0, it won't be taken into calculation.
# def cossim_dog_dog(dogx_index,dogy_index):
#     listx = make_list(dogx_index,hard_constraints=True,tfidf=True,soft_constraints=True)
#     listy = make_list(dogy_index,hard_constraints=True,tfidf=True,soft_constraints=True)
#     for i in range(1773,1779):
#         if listx[i] == 0 or listy[i] == 0:
#             listx[i] = 0
#             listy[i] = 0
#     cossim = cosine_similarity(listx,listy)
#     return cossim

In [12]:
# # Calculation of cosine similarity between one dog and the user's profile
# # Used in recommendation for a new user
# # The hard constraints are filtered before the calcularion, thus not taken into account
# # For the soft constraints, if the value of one certain aspect for the dog is 0, 
# # or that of the user's profile is 1, it won't be taken into calculation.
# def cossim_dog_userprofile(user,dog_index):
#     listx = make_list(user,False,True,True)
#     listy = make_list(dog,False,True,True)
#     for i in range(1773-3,1779-3):
#         if listx[i] == 1 or listy[i] == 0:
#             listx[i] = 0
#             listy[i] = 0
#     cossim = cosine_similarity(listx,listy)
#     return cossim

# # Test
# # cossim_dog_dog(data.loc[0],data.loc[100])

In [13]:
# # Calculation of cosine similarity between one dog and the user's history selection
# # Used in recommendation for an old user after he/she has selected some dogs
# # Only consider the descriptions of the dogs
# def cossim_dog_userhistory(history,dog_index):
# #     listy = make_list(dog_index,hard_constraints=False,tfidf=True,soft_constraints=False,data)
#     listy = make_list(dog_index,False,True,False)
#     cossim = cosine_similarity(history,listy)
#     return cossim

In [14]:
# Based on user's history data, build a user's prefernce.
# The return is a list
def user_history(data,dogs_index):
    history = []
    history.append(999)
    history.append('History')
    history.append(0)
    history.append(0)
    history.append(0)
    history.append('courpus')
    for i in range(6):
        history.append(0)
    for i in range(1770):
        summ = 0
        average = 0
        for index in dogs_index:
            summ += data.loc[index]['tfidf'+str(i)]
        average = summ/len(dogs_index)
        history.append(average)
#     data.loc[335] = history
    return history

## Recommender for old users based on the user's history prefernece

In [15]:
# Find the similar dogs according to cosine similarity
def recommend_userhistory(data,dogs_filter_index,history_index,target_num):
    similarity = []
    history = user_history(data,history_index)
    data.loc[999] = history
    print('History preference made')
#     num = 1
    for index in history_index:
        if index in dogs_filter_index:
#             print(index)
            dogs_filter_index.remove(index)
    for index in dogs_filter_index:
        cos_sim = cosine_similarity(data.loc[index],data.loc[999],hard_constraint=False,soft_constraint=False,tfidf=True,user=False)
#         print(str(num) + '/' + str(len(dogs_filter_index)) + ' : ' + str(cos_sim))
#         num += 1
        similarity.append(cos_sim)
#     print(similarity)
    max_index_ = list(map(similarity.index, heapq.nlargest(target_num,similarity)))
    max_index = []
    for i in range(len(max_index_)):
        max_index.append(dogs_filter_index[max_index_[i]])
    max_value = heapq.nlargest(target_num,similarity)
    return [[max_index],[max_value]]

def filter_dont_like(index_old,dont_like):
    for index in dont_like:
        if index in index_old:
            index_old.remove(index)
    return index_old

In [16]:
filtered_index = filter_dog(data,[0,1],[3],[0])
dont_like_index = [3,5,6] #These are indexes for dogs that the user don't like
filtered_index_remove_dontlike = filter_dont_like(nofilter, dont_like_index)
# No filter means all the dogs are picked. No need to do the filter for age gender and HDB.
# filtered_index = filter_dont_like(filtered)
# print(filtered_index)
history_index = [0,20,42,25] #These are indexes for dogs that the user like
recommend_dogs = recommend_userhistory(data,nofilter,history_index,10)
print(recommend_dogs)

History preference made
[[[198, 39, 160, 96, 146, 159, 65, 186, 81, 320]], [[0.3011773531208616, 0.22443746451740343, 0.21160883484017914, 0.20499462197300167, 0.2018001937719968, 0.19731779227591595, 0.1954100313205122, 0.18942741368501, 0.18253075331013463, 0.18025086571119484]]]


In [18]:
print('History Preference Dogs:')
for index in history_index:
    print(data.loc[index]['Corpus'])
    
print('Recommended Dogs:')
for index in recommend_dogs[0][0]:
    print(data.loc[index]['Corpus'])

History Preference Dogs:
 absolute darling food take medicine food aggression starve point confident trust independent separation anxiety company story able smiley face
 stunning little settle training socialisation
 shy sweet quiet sweet affectionate super shy face guess stop share good guidance support eat play walk sleep trust smart bad
 giant affection excitement welcome happily enter enclosure walk leash outside spook vehicle panic hard size strong able walk sweet oversized affection attention
Recommended Dogs:
 quiet super sweet little shy good food motivate walk leash
 shy reserved smart special enjoy quiet nervous confidence take walk outside leash stop walk sweet kind play good hassle
 little shy quiet calm walk leash
 quiet shy sweet gorgeous affectionate kind share son company gentle confident cuddle enjoy walk outdoor adventure walk leash pace calm support senior
 cute little puppy aggression settle little fearful insecure pant sweet little underneath stick comfort sit encl

## **Recommender for new users**

In [19]:
def process_user_description(description, corpus_words):
    des = ''
    sentence = my_preprocessing(description)
#     print(sentence)
    for word in sentence:
        if word in corpus_words:
            des += ' '
            des += word
#     print(des)
    return des

def new_user_tfidf_list(des,corpus):
    corpus_new = []
    corpus_new.append(des)
    for text in corpus:
        corpus_new.append(text)
#     print(len(corpus_new))
    vectorizer = CountVectorizer()
    word_vec = vectorizer.fit_transform(corpus_new)
    transformer = TfidfTransformer()
    tfidf = transformer.fit_transform(word_vec)
    tfidf_matrix = tfidf.toarray()
#     print(tfidf_matrix.shape)
    return tfidf_matrix[0]
    
des = process_user_description('feh ge cute fe afe',corpus_words)
len(new_user_tfidf_list(des,corpus))

1770

In [20]:
def build_new_user(description,tags,corpus,corpus_words):
    user = []
    user.append(666)
    user.append('NewUser')
    user.append(0)
    user.append(0)
    user.append(0)
    des = process_user_description(description, corpus_words)
#     print(des)
    user.append(des)
    for i in tags:
        user.append(i)
    for i in new_user_tfidf_list(des,corpus):
        user.append(i)
    return user

# data.loc[666] = build_new_user('cute lovely shy quiet',[1,1,1,1,1,1],corpus,corpus_words)

In [24]:
def recommend_newuser(data,gender,age,hdb,description,tags,corpus,corpus_words,target_num,filter_hard):
#     print(description)
#     print(tags)
    data.loc[666] = build_new_user(description,tags,corpus,corpus_words)
    if filter_hard == True:
        filtered_index = filter_dog(data,gender,age,hdb)
    else:
        filtered_index = filter_dog(data,[0,1],[0,1,2,3],[0,1])
    similarity = []
    print('Dogs after filtering:' + str(filtered_index))
    for index in filtered_index:
        cos_sim = cosine_similarity(data.loc[index],data.loc[666],hard_constraint=False,soft_constraint=True,tfidf=True,user=True)
        similarity.append(cos_sim)
    for i in range(len(similarity)):
        print('similarity for index ' + str(filtered_index[i])+ ' : ' +str(similarity[i]))
    max_index_ = list(map(similarity.index, heapq.nlargest(target_num,similarity)))
    max_index = []
    num = min(target_num,len(filtered_index))
    for i in range(len(max_index_)):
        max_index.append(filtered_index[max_index_[i]])
    max_value = heapq.nlargest(num,similarity)
#     print(len(max_index))
#     max_value = [0,0,0,0,0] #try all 0
    return_null = 1
    for i in range(num):
        if max_value[i] != 0:
            return_null = 0
    if return_null == 1:
        return []
    for i in range(num):
        if max_value[num-i-1] == 0:
#             print(len(max_index)-i-1)
            max_index.pop(num-i-1)
            max_value.pop(num-i-1)
    return max_index


In [31]:
max_index = recommend_newuser(data,[1],[2],[1],'ute',[1,1,1,1,1,1],corpus,corpus_words,5,filter_hard=True)
# print()
print(max_index)
# print(max_value)

Dogs after filtering:[24, 45, 97, 108, 138, 144, 161, 171, 177, 182, 190, 192, 195, 217, 227, 260, 280, 285, 320]
similarity for index 24 : 0
similarity for index 45 : 0
similarity for index 97 : 0
similarity for index 108 : 0
similarity for index 138 : 0
similarity for index 144 : 0
similarity for index 161 : 0
similarity for index 171 : 0
similarity for index 177 : 0
similarity for index 182 : 0
similarity for index 190 : 0
similarity for index 192 : 0
similarity for index 195 : 0
similarity for index 217 : 0
similarity for index 227 : 0
similarity for index 260 : 0
similarity for index 280 : 0
similarity for index 285 : 0
similarity for index 320 : 0
[]


In [160]:
print('Dogs Recommended:')
for index in max_index[0]:
    print(data.loc[index]['Corpus'])
    tags = []
    for i in range(6):
        tags.append(data.loc[index]['Tag'+str(i+3)])
    print(tags)

Dogs Recommended:
 cute small walk leash
[0, 0, 0, 0, 0, 0]
 shy warm affectionate ask quiet walk rub
[0, 0, 0, 0, 0, 0]
 quiet super sweet little shy good food motivate walk leash
[0, 0, 0, 0, 0, 0]
 shy sweet quiet sweet affectionate super shy face guess stop share good guidance support eat play walk sleep trust smart bad
[0, 0, 0, 0, 0, 0]
 development stage suffice cute smart presence puppy young term good
[0, 0, 0, 0, 0, 1]
 good shy indoor avs catcher petrify take warm quiet gentle calm nature leash walk willing indoor walk leash
[0, 0, 0, 0, 0, 1]
 loyal friendly nervous surrender lovely loyal anxious surrender allegedly bite suggest bit possessive child behaviour walk behaviour result frightened loud happen outside patient resettle
[0, 0, 0, 0, 1, 0]
 confident dare explore afraid photo angel cute choose mean peace safety italian lord enable save
[0, 0, 0, 0, 0, 1]
 shy warm food motivated peepad aggression walk leash long settle
[0, 0, 0, 0, 0, 0]
 approx outram long terrified